# IT Academy - Data Science amb Python  
## Task 7: Log Transformation with Regular Expressions  
### [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)  

[![forthebadge made-with-python](http://ForTheBadge.com/images/badges/made-with-python.svg)](https://www.python.org/)  
[![Made withJupyter](https://img.shields.io/badge/Made%20with-Jupyter-orange?style=for-the-badge&logo=Jupyter)](https://jupyter.org/try)  
[![Code Time](https://wakatime.com/badge/github/jesussantana/Registre_de_logs.svg)](https://wakatime.com/badge/github/jesussantana/Registre_de_logs) 

### Exercise 1
- Normalize, identify, and enumerate each of the attributes / variables in the "Web_access_log-akumenius.com" file structure found in the GitHub repository "Data-sources".

In [1]:
import multiprocessing
import numpy as np
import pandas as pd
import warnings
import requests
import json
import time
import re

import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import seaborn as sns
%matplotlib inline

from datetime import datetime
from joblib import Parallel, delayed
from pandas import json_normalize
from device_detector import SoftwareDetector

warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

path = 'data/Web_access_log-akumenius.com.txt'

Logs_raw = pd.read_csv(path, sep='\s | \- | \"', names =['DNS','ip','Date','Time','Request','Status','Size','Referer','UserAgent'], engine='python')

Logs_copy = Logs_raw.copy()

Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
1,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
2,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
3,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
4,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN


- We load the data to check how it has been distributed
  - The variables that we will use:
    - 'DNS','IP','Date','Time','Request','Status','Size','Referer','UserAgent'

###  Exercise 2
- Cleans, preprocesses, structures and transforms (dataframe) the data of the Web Access record.

- Check rows and columns

In [3]:
Logs_copy.tail()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
261868,www.akumenius.com 5.255.253.53,- [02/Mar/2014:03:05:39 +0100],"GET / HTTP/1.1"" 200 7528","-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,None,None,None,NaN
261869,www.akumenius.com 74.86.158.107,- [02/Mar/2014:03:09:52 +0100],"HEAD / HTTP/1.1"" 200","""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,None,None,None,NaN
261870,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
261871,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
261872,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN


In [4]:
Logs_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261873 entries, 0 to 261872
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   DNS        261873 non-null  object 
 1   ip         261873 non-null  object 
 2   Date       261873 non-null  object 
 3   Time       261873 non-null  object 
 4   Request    261873 non-null  object 
 5   Status     486 non-null     object 
 6   Size       1 non-null       object 
 7   Referer    1 non-null       object 
 8   UserAgent  0 non-null       float64
dtypes: float64(1), object(8)
memory usage: 18.0+ MB


In [5]:
Logs_copy.shape

(261873, 9)

- Check for null data

In [6]:
Logs_copy.isnull().sum()

DNS               0
ip                0
Date              0
Time              0
Request           0
Status       261387
Size         261872
Referer      261872
UserAgent    261873
dtype: int64

In [7]:
Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
1,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
2,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
3,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
4,localhost 127.0.0.1,- [23/Feb/2014:03:10:31 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN


In [8]:
Logs_copy.tail()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
261868,www.akumenius.com 5.255.253.53,- [02/Mar/2014:03:05:39 +0100],"GET / HTTP/1.1"" 200 7528","-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,None,None,None,NaN
261869,www.akumenius.com 74.86.158.107,- [02/Mar/2014:03:09:52 +0100],"HEAD / HTTP/1.1"" 200","""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,None,None,None,NaN
261870,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
261871,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN
261872,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",None,None,None,NaN


- Reorder columns of data

In [9]:
Logs_copy.UserAgent = Logs_copy.Request
Logs_copy.Request = Logs_copy.Date
Logs_copy.Referer = Logs_copy.Time
Logs_copy.Date = Logs_copy.ip
Logs_copy.Time = Logs_copy.ip = None

Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost 127.0.0.1,None,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost 127.0.0.1,None,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost 127.0.0.1,None,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost 127.0.0.1,None,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost 127.0.0.1,None,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


- DNS & IP data split

In [10]:
Logs_copy[['DNS','ip']] = Logs_copy.DNS.str.split('\s', expand = True).get([0, 1])

In [11]:
Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],None,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Check how many different DNS & Ips exist

In [12]:
Logs_copy.DNS.unique()

array(['localhost', 'www.akumenius.com', 'akumenius.com', 'akumenius.es',
       'test.akumenius.com'], dtype=object)

In [13]:
len(Logs_copy.ip.unique())

2921

- Normalize DNS

In [14]:
String_pattern = "^[aeiouAEIOU]{1}[A-Za-z]*"
Logs_copy.DNS = Logs_copy.DNS.apply(lambda x: (('www.{}'.format(x)) if re.search(String_pattern, x) else x))

In [15]:
Logs_copy.DNS.unique()

array(['localhost', 'www.akumenius.com', 'www.akumenius.es',
       'test.akumenius.com'], dtype=object)

- Time data extraction

In [16]:
Logs_copy.Time =pd.to_datetime(Logs_copy.Time,format='%d/%b/%Y:%H:%M:%S %z',errors='ignore')


Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],NaT,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],NaT,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],NaT,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],NaT,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],NaT,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


In [17]:
Logs_copy.Time = Logs_copy.Date.str.extract(':(\d{2}:\d{2}:\d{2}.*)]')
Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,- [23/Feb/2014:03:10:31 +0100],03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Date data extraction

In [18]:
Logs_copy.Date = Logs_copy.Date.str.extract('(\d+/\w+/\d+)')

Logs_copy.Date = pd.to_datetime(Logs_copy.Date, format = '%d/%b/%Y')

In [19]:
Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,"OPTIONS * HTTP/1.0"" 200",None,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Request & Status data split

In [20]:
Logs_copy[['Request','Status']] = Logs_copy.Request.str.split('\"', expand = True).get([0, 1])

In [21]:
Logs_copy.tail()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
261868,www.akumenius.com,5.255.253.53,2014-03-02,03:05:39 +0100,GET / HTTP/1.1,200 7528,None,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02,03:09:52 +0100,HEAD / HTTP/1.1,200,None,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
261871,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
261872,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


In [22]:
Logs_copy.head()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,None,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Size data extraction

In [23]:
Logs_copy.Size = Logs_copy.Status.str.extract('(\d+$)')
Logs_copy

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
...,...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,2014-03-02,03:05:39 +0100,GET / HTTP/1.1,200 7528,7528,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02,03:09:52 +0100,HEAD / HTTP/1.1,200,200,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
261871,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Clean Size data

In [24]:
%%time

Logs_copy.Size = Logs_copy.Size.apply(lambda x: (np.nan if x == '200' else x))

CPU times: user 87.4 ms, sys: 95 µs, total: 87.5 ms
Wall time: 85.9 ms


In [25]:
Logs_copy

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
...,...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,2014-03-02,03:05:39 +0100,GET / HTTP/1.1,200 7528,7528,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02,03:09:52 +0100,HEAD / HTTP/1.1,200,NaN,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
261871,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Status data extraction

In [26]:
Logs_copy.Status = Logs_copy.Status.str.extract('(\d{3})')
Logs_copy.tail()

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
261868,www.akumenius.com,5.255.253.53,2014-03-02,03:05:39 +0100,GET / HTTP/1.1,200,7528,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02,03:09:52 +0100,HEAD / HTTP/1.1,200,NaN,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
261871,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
261872,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"


- Clean Referer data

In [27]:
%%time
Logs_copy.Referer = Logs_copy.Referer.astype(str)
Logs_copy.Referer = Logs_copy.Referer.apply(lambda x: (np.nan if re.search('-"', x) else x.rstrip('-"')))

CPU times: user 310 ms, sys: 48 µs, total: 310 ms
Wall time: 308 ms


In [28]:
Logs_copy.Referer.unique()

array([nan,
       'http://www.akumenius.com/hoteles-baratos/ofertas-hotel-Playa-Blanca-All-Inclusive-en-Cayo%20Largo_CUBA-62359b.html',
       'http://www.akumenius.com/', ...,
       '"http://www.akumenius.com/booking/152186/t/BARCELO%20DOMINICAN%20BEACH/4',
       'http://www.akumenius.com/hotelDetailPopup/49612/b',
       'http://www.akumenius.com/destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Phuket_TAILANDIA-con-Windsurf'],
      dtype=object)

In [29]:
Logs_copy

,DNS,ip,Date,Time,Request,Status,Size,Referer,UserAgent
0,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
1,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
2,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
3,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
4,localhost,127.0.0.1,2014-02-23,03:10:31 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
...,...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,2014-03-02,03:05:39 +0100,GET / HTTP/1.1,200,7528,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02,03:09:52 +0100,HEAD / HTTP/1.1,200,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"
261871,localhost,127.0.0.1,2014-03-02,03:10:18 +0100,OPTIONS * HTTP/1.0,200,NaN,NaN,"Apache (internal dummy connection)"" VLOG=-"


###  Exercise 3
- Geolocates IP's. Here is a page of interest:
  - [freegeoip](https://freegeoip.app/)

- We export Ips file for security

In [30]:
Ips_export= Logs_copy.ip.copy()

Ips_export.replace('', 'null', inplace = True)

Ips_export.to_csv('../../Data/Ips_export.csv', index = False)

- Ips file recovery

In [31]:
path = '../../Data/Ips_export.csv'

Ips_raw = pd.read_csv(path, sep= 'delimiter', engine='python')

Ips_raw.head()

,ip
0,127.0.0.1
1,127.0.0.1
2,127.0.0.1
3,127.0.0.1
4,127.0.0.1


In [32]:
Ips_raw.tail()

,ip
261868,5.255.253.53
261869,74.86.158.107
261870,127.0.0.1
261871,127.0.0.1
261872,127.0.0.1


- Make a copy of the data to be used and we check them

In [33]:
ips_unique = pd.DataFrame({"ip": Ips_raw.ip.unique()})

In [34]:
len(ips_unique)

2921

In [35]:
ips_unique

,ip
0,127.0.0.1
1,66.249.76.216
2,5.255.253.53
3,157.55.35.112
4,193.111.141.116
...,...
2916,46.222.83.180
2917,79.155.78.117
2918,217.174.248.179
2919,188.135.173.80


In [36]:
url = "https://freegeoip.app/json/"
localhost = "127.0.0.1"
local = {'ip': '127.0.0.1',
 'country_code': 'ES',
 'country_name': 'Spain',
 'region_code': 'GI',
 'region_name': 'Girona',
 'city': 'Girona',
 'zip_code': '17003',
 'time_zone': 'Europe/Madrid',
 'latitude': 41.9831,
 'longitude': 	2.8249,
 'metro_code': 0}

- Function for extract Information freegeoip

In [37]:
def extract_info(ip):
    if ip != localhost:
        try:
            response = requests.get(url+ip)
            if response.status_code == 200:
                return response.json()
        except TypeError:
            pass       
    else:
        return local

- Ips Information Extraction

In [ ]:
%%time

geolocations = {}

geolocations = (Parallel(n_jobs = 8, backend = "multiprocessing")(map(delayed(extract_info), ips_unique.ip)))

len(geolocations)

- joblib.Parallel uses the backend module to start worker processes, executing tasks simultaneously on separate CPUs.
- It takes about 1 minute to extract the data from the 2921 IPS, much faster when using 4 cores and 8 threads

- Check the Data obtained

In [ ]:
type(geolocations)

In [ ]:
geolocations[1]

- Convert to Dataframe

In [ ]:
geolocation_df = pd.DataFrame(geolocations)

In [ ]:
geolocation_df

- Export Geolocations to JSON File

In [ ]:
geolocation_df_export = geolocation_df.copy()
geolocation_df_export.to_json('data/geolocation.json', orient="records")

-  Export Geolocations to CSV File

In [ ]:
geolocation_df_export = geolocation_df.copy()

geolocation_df_export.replace('', 'null', inplace = True)

geolocation_df_export.to_json('data/geolocation.csv')

In [ ]:
geolocation_df.info()

- Do a merge by the column of IPs

In [ ]:
Logs_copy2 = Logs_copy.merge(geolocation_df, on='ip', how='outer')

- Check the result

In [ ]:
Logs_copy2.tail()

In [ ]:
Logs_copy2.metro_code.unique()

In [ ]:
Logs_copy2.info()

###  Exercise 4
- Show me your creativity, Surprise me go one step further with the above analysis.

- Extract UserAgent Data

In [ ]:
%%time

Devices = Logs_copy2.UserAgent

device = Devices.apply(lambda x: SoftwareDetector(x).parse())

print(device)

- Assign the values

In [ ]:
%%time

Logs_copy2['Client_Name'] = device.apply(lambda x: x.client_name())
Logs_copy2['Client_Type'] = device.apply(lambda x: x.client_type())
Logs_copy2['Client_Version'] = device.apply(lambda x: x.client_version())
Logs_copy2['Os_Name'] = device.apply(lambda x: x.os_name())
Logs_copy2['Os_Version'] = device.apply(lambda x: x.os_version())
Logs_copy2['Engine'] = device.apply(lambda x: x.engine())
Logs_copy2['Device_Brand_Name'] = device.apply(lambda x: x.device_brand_name())
Logs_copy2['Device_Model'] = device.apply(lambda x: x.device_model())
Logs_copy2['Device_Type'] = device.apply(lambda x: x.device_type())



- Rename & Reorder columns

In [ ]:
Logs_copy2.columns = map(str.upper, Logs_copy2.columns)
Logs_copy2.columns

In [ ]:
Logs_copy2 = Logs_copy2[['DNS', 'IP', 'DATE', 'TIME','COUNTRY_CODE', 'COUNTRY_NAME', 'REGION_CODE',
       'REGION_NAME', 'CITY', 'ZIP_CODE', 'TIME_ZONE', 'LATITUDE', 'LONGITUDE','METRO_CODE', 'REQUEST', 'STATUS', 'SIZE', 'REFERER','USERAGENT','CLIENT_NAME', 'CLIENT_TYPE', 'CLIENT_VERSION','OS_NAME','OS_VERSION','ENGINE', 'DEVICE_BRAND_NAME', 'DEVICE_MODEL','DEVICE_TYPE']]
Logs_copy2.head()

- Check the result

In [ ]:
Logs_copy2.tail()

In [ ]:
Logs_copy2.USERAGENT[0]

- The column from which we have extracted the data is deleted

In [ ]:
del(Logs_copy2['USERAGENT'])

Logs_copy2.tail()

- Clean Client Data

In [ ]:
%%time

Logs_copy2.CLIENT_NAME = Logs_copy2.CLIENT_NAME.apply(lambda x: (x[0:6] if re.search('Apache', x) else x))

In [ ]:
Logs_copy2

- Number of user views by DNS

In [ ]:
sns.set_theme(style="darkgrid")


In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 8),sharex=True,tight_layout=True)

dns = sns.barplot(data=Logs_copy2.groupby('DNS')[['IP']].count().sort_values(by='IP',ascending=False).reset_index().head(10),
                    y='DNS',  
                    x='IP',
                    palette='mako_r',
                    ax=ax1)

dns.set(title='Number of user views by DNS',
          ylabel=None,
          xlabel=None)

- Number of user views by Country

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 8),sharex=True,tight_layout=True)

country = sns.barplot(data=Logs_copy2.groupby('COUNTRY_NAME')[['IP']].count().sort_values(by='IP',ascending=False).reset_index().head(10),
                    y='COUNTRY_NAME',  
                    x='IP',
                    palette='rocket_r',
                    ax=ax1)

country.set(title='Number of user views by country',
          ylabel=None,
          xlabel=None)

- Number of user views by City

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 8),sharex=True,tight_layout=True)

city = sns.barplot(data=Logs_copy2.groupby('CITY')[['IP']].count().sort_values(by='IP',ascending=False).reset_index().head(10),
                    y='CITY',  
                    x='IP',
                    palette='viridis_r',
                    ax=ax1)

city.set(title='Number of user views by City',
          ylabel=None,
          xlabel=None)

- Number of user views by Request

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 8),sharex=True,tight_layout=True)

city = sns.barplot(data=Logs_copy2.groupby('REQUEST')[['IP']].count().sort_values(by='IP',ascending=False).reset_index().head(10),
                    y='REQUEST',  
                    x='IP',
                    palette='inferno_r',
                    ax=ax1)

city.set(title='Number of user views by Request',
          ylabel=None,
          xlabel=None)

- Number of user views by Referer

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 8),sharex=True,tight_layout=True)

city = sns.barplot(data=Logs_copy2.groupby('REFERER')[['IP']].count().sort_values(by='IP',ascending=False).reset_index().head(10),
                    y='REFERER',  
                    x='IP',
                    palette='magma_r',
                    ax=ax1)

city.set(title='Number of user views by Referer',
          ylabel=None,
          xlabel=None)

- views by Device and OS

In [ ]:

sns.displot(data = Logs_copy2, x = "DATE", hue = "OS_NAME", multiple="stack", kind="kde", height=10)

- Status by number of views

In [ ]:
graph = Logs_copy2[['STATUS', 'DNS']].groupby(['STATUS']).count().sort_values(by = 'DNS',ascending = False)
graph = graph.rename(columns = {'DNS' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'r', ylabel = 'Frequency', legend = None, figsize = (14,7))
plt.show()

- Views by Client Type

In [ ]:
graph = Logs_copy2[['CLIENT_TYPE', 'DNS']].groupby(['CLIENT_TYPE']).count().sort_values(by = 'DNS',ascending = False)
graph = graph.rename(columns = {'DNS' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'g', ylabel = 'Frequency', legend = None, figsize = (14,7))
plt.show()

- Device type for DNS

In [ ]:
sns.set(rc={"figure.figsize": (20, 10)})
sns.displot(data = Logs_copy2, x = "DATE", hue = "DEVICE_TYPE", multiple = "stack", kind="kde", height=10)

- Geolocations

In [ ]:
GEOLOCATIONS = Logs_copy2[Logs_copy2['LONGITUDE'] != " "][['IP', 'LONGITUDE', 'LATITUDE']].astype({'LONGITUDE': float,'LATITUDE': float})
GEOLOCATIONS = (GEOLOCATIONS[['IP', 'LONGITUDE', 'LATITUDE']].groupby('IP').agg({'LONGITUDE' : np.mean, 'LATITUDE': np.mean, 'IP' : 'count'}).astype({'IP': float}))

GEOLOCATIONS.info()

- Geolocations of web Views

In [ ]:
def main():
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    
    ax.set_global()

    ax.stock_img()
    ax.coastlines()
   
    ax.plot(GEOLOCATIONS.LONGITUDE, GEOLOCATIONS.LATITUDE, '*', alpha= 0.15, transform=ccrs.PlateCarree(), color='darkgreen')

    plt.title('Geolocations of web Views',fontdict={'family': 'serif', 
                    'color' : 'black',
                    'weight': 'bold',
                    'size': 18})
    plt.show()


if __name__ == '__main__':
    main()


- Zone more density of Views to the web

In [ ]:
def main():
    fig = plt.figure(figsize=(30, 15))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
    


    central_lon, central_lat = -3.7492, 40.4636
    extent = [-20, 20, 30, 55]
    ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
    ax.set_extent(extent)
    ax.stock_img()
    ax.coastlines(resolution='50m')
   
    ax.plot(GEOLOCATIONS.LONGITUDE, GEOLOCATIONS.LATITUDE, '*', alpha= 0.15, transform=ccrs.PlateCarree(), color='green')

    plt.title('Zone more density of web iews',fontdict={'family': 'serif', 
                    'color' : 'black',
                    'weight': 'bold',
                    'size': 18})
    plt.show()


if __name__ == '__main__':
    main()

- Save the data obtained for later reuse

In [ ]:
Logs_export = Logs_copy2.copy()

Logs_export.replace('', 'null', inplace = True)

Logs_export.to_csv('data/Logs_export.csv', index = False)

## Build Dashboard In Python – Plotly Dash 
### Dash is a Python framework, based on Flask, Plotly and ReactJS
### New environment for the project v0.1

- mkdir web_log && cd web_log
- python3 -m venv venv
- source venv/bin/activate

- python -m pip install dash==1.13.3 pandas==1.0.5

### Create app.py Dashboard

### Run Application

- python app.py

### [Web Log Dash App Github Repo](https://github.com/jesussantana/web-log-dash)
### [Web Log Dash App Deploy](http://jesussantana.pythonanywhere.com/)